In [1]:
import os
import datetime
from datetime import timedelta
import quandl
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import plotnine as p9
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import zscore
import pandas_market_calendars as mcal

import warnings
import functools
import wrds
import sys
# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
db = wrds.Connection()


WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [3]:
earnings_dates_for_tickers = pd.read_csv("earnings_dates_for_tickers.csv")
earnings_dates_for_tickers

,Unnamed: 0,index,Short_ticker,Name,Ticker,Date,Event Type,Adjusted Trading Date,trade_entry,trade_exit
0,0,396,A,Agilent Technologies Inc,A US,2018-02-14 16:05:00,ER,2018-02-15 00:00:00+00:00,2018-02-14 16:05:00,2018-02-15 16:05:00
1,1,1298,A,Agilent Technologies Inc,A US,2018-05-14 16:05:00,ER,2018-05-15 00:00:00+00:00,2018-05-14 16:05:00,2018-05-15 16:05:00
2,2,2023,A,Agilent Technologies Inc,A US,2018-08-14 16:05:00,ER,2018-08-15 00:00:00+00:00,2018-08-14 16:05:00,2018-08-15 16:05:00
3,3,2754,A,Agilent Technologies Inc,A US,2018-11-19 16:05:00,ER,2018-11-20 00:00:00+00:00,2018-11-19 16:05:00,2018-11-20 16:05:00
4,4,3297,A,Agilent Technologies Inc,A US,2019-02-20 16:05:00,ER,2019-02-21 00:00:00+00:00,2019-02-20 16:05:00,2019-02-21 16:05:00
...,...,...,...,...,...,...,...,...,...,...
19471,19471,15969,ZTS,Zoetis Inc,ZTS US,2023-02-14 07:00:00,ER,2023-02-13 00:00:00+00:00,2023-02-13 07:00:00,2023-02-14 07:00:00
19472,19472,16912,ZTS,Zoetis Inc,ZTS US,2023-05-04 07:00:00,ER,2023-05-03 00:00:00+00:00,2023-05-03 07:00:00,2023-05-04 07:00:00
19473,19473,17936,ZTS,Zoetis Inc,ZTS US,2023-08-08 07:00:00,ER,2023-08-07 00:00:00+00:00,2023-08-07 07:00:00,2023-08-08 07:00:00
19474,19474,18652,ZTS,Zoetis Inc,ZTS US,2023-11-02 07:00:00,ER,2023-11-01 00:00:00+00:00,2023-11-01 07:00:00,2023-11-02 07:00:00


In [4]:
csv_file_path = "secids.csv"

# Reading the CSV file into a NumPy array
secids = np.genfromtxt(csv_file_path, delimiter=',')
secids

array([101149., 205456., 210354., ..., 211262., 207854., 105959.])

In [5]:
options_dict = {} 
for year in [2018]:
    print(year)
    table_name = f"optionm.opprcd{year}"  # Generate table name dynamically
    query = f"""
    SELECT
        date, secid, symbol, cp_flag, expiry_indicator, volume, strike_price, exdate, open_interest, impl_volatility, best_bid, best_offer, delta, gamma, vega, theta
    FROM 
        {table_name} a 
    WHERE
        a.secid IN ({','.join(map(str, secids))}) AND
        a.volume > 0 AND
        a.exdate - a.date <= 38
    """
    options_dict[year] = db.raw_sql(query, date_cols=['date'])

2018


In [6]:
for year in range(2018, 2019):
    # Get the DataFrame for the current year
    options = options_dict[year]
    
    # Split the 'symbol' column and extract the first substring as 'ticker'
    options['ticker'] = options['symbol'].str.split(' ').str[0]
    
    # Set the multi-index using 'date' and 'ticker'
    options.set_index(['ticker','date', 'exdate'], inplace=True)
    
    # Update the options_dict with the modified DataFrame
    options_dict[year] = options

In [7]:
options_dict[2018]

secid            symbol cp_flag  \
ticker date       exdate                                           
FTS    2018-01-09 2018-02-16    6505.0  FTS 180216C35000       C   
       2018-01-11 2018-02-16    6505.0  FTS 180216C35000       C   
       2018-01-12 2018-02-16    6505.0  FTS 180216C35000       C   
       2018-01-16 2018-02-16    6505.0  FTS 180216C35000       C   
       2018-01-17 2018-02-16    6505.0  FTS 180216C35000       C   
...                                ...               ...     ...   
ANF    2018-09-28 2018-11-02  100974.0  ANF 181102C20000       C   
                  2018-11-02  100974.0  ANF 181102C26000       C   
                  2018-11-02  100974.0  ANF 181102P20000       P   
                  2018-11-02  100974.0  ANF 181102P21000       P   
                  2018-11-02  100974.0  ANF 181102P23000       P   

                             expiry_indicator  volume  strike_price  \
ticker date       exdate                                              
FTS    2018-01-09 2018-02-16             None    10.0       35000.0   
       2018-01-11 2018-02-16             None     7.0       35000.0   
       2018-01-12 2018-02-16             None    10.0       35000.0   
       2018-01-16 2018-02-16             None    62.0       35000.0   
       2018-01-17 2018-02-16             None    19.0       35000.0   
...                                       ...     ...           ...   
ANF    2018-09-28 2018-11-02                w     1.0       20000.0   
                  2018-11-02                w     1.0       26000.0   
                  2018-11-02                w     1.0       20000.0   
                  2018-11-02                w    38.0       21000.0   
                  2018-11-02                w     5.0       23000.0   

                              open_interest  impl_volatility  best_bid  \
ticker date       exdate                                                 
FTS    2018-01-09 2018-02-16           10.0         0.096782      0.65   
       2018-01-11 2018-02-16           20.0         0.172833      0.15   
       2018-01-12 2018-02-16           20.0         0.156504      0.00   
       2018-01-16 2018-02-16           20.0         0.121353      0.20   
       2018-01-17 2018-02-16           82.0         0.219486      0.55   
...                                     ...              ...       ...   
ANF    2018-09-28 2018-11-02            4.0         0.466629      1.84   
                  2018-11-02            2.0         0.438565      0.07   
                  2018-11-02           32.0         0.472242      0.69   
                  2018-11-02           50.0         0.455927      1.09   
                  2018-11-02           10.0         0.460479      2.25   

                              best_offer     delta     gamma      vega  \
ticker date       exdate                                                 
FTS    2018-01-09 2018-02-16        1.60  0.833393  0.228592  2.662240   
       2018-01-11 2018-02-16        1.40  0.525980  0.214809  4.330575   
       2018-01-12 2018-02-16        1.30  0.504768  0.241263  4.247554   
       2018-01-16 2018-02-16        0.80  0.521278  0.332326  3.956877   
       2018-01-17 2018-02-16        1.45  0.562202  0.182695  3.932497   
...                                  ...       ...       ...       ...   
ANF    2018-09-28 2018-11-02        1.87  0.678862  0.117316  2.341548   
                  2018-11-02        0.11  0.074037  0.048891  0.915791   
                  2018-11-02        0.72 -0.323297  0.116547  2.347388   
                  2018-11-02        1.12 -0.450972  0.133343  2.588441   
                  2018-11-02        2.51 -0.698382  0.117426  2.276175   

                                 theta  
ticker date       exdate                
FTS    2018-01-09 2018-02-16 -1.861654  
       2018-01-11 2018-02-16 -4.215676  
       2018-01-12 2018-02-16 -3.875076  
       2018-01-16 2018-02-16 -3.277378  
       2018-01-17 2018-02-16 -5.757857  
...        

In [8]:
stock_prices = pd.read_parquet("stock_price_ED_20182023.parquet")
stock_prices

,date,permno,prc,bid,ask,ticker,comnam,permco
0,2018-02-07,87432,68.06,68.06000,68.07000,A,AGILENT TECHNOLOGIES INC,36364.0
1,2018-02-08,87432,65.05,65.03000,65.05000,A,AGILENT TECHNOLOGIES INC,36364.0
2,2018-02-09,87432,66.98,66.97000,66.98000,A,AGILENT TECHNOLOGIES INC,36364.0
3,2018-02-12,87432,68.43,68.40000,68.41000,A,AGILENT TECHNOLOGIES INC,36364.0
4,2018-02-13,87432,68.34,68.33000,68.34000,A,AGILENT TECHNOLOGIES INC,36364.0
...,...,...,...,...,...,...,...,...
147109,2023-11-03,13788,162.23,162.19000,162.23000,ZTS,ZOETIS INC,54327.0
147110,2023-11-06,13788,163.17,163.20000,163.27000,ZTS,ZOETIS INC,54327.0
147111,2023-11-07,13788,167.16,167.24001,167.28000,ZTS,ZOETIS INC,54327.0
147112,2023-11-08,13788,170.87,170.89000,170.92999,ZTS,ZOETIS INC,54327.0


In [9]:
earnings_dates_for_tickers['trade_entry'] = pd.to_datetime(earnings_dates_for_tickers['trade_entry'])

# Extract only the date part (YYYY-MM-DD)
earnings_dates_for_tickers['trade_entry'] = earnings_dates_for_tickers['trade_entry'].dt.date
earnings_dates_for_tickers['trade_entry']

0        2018-02-14
1        2018-05-14
2        2018-08-14
3        2018-11-19
4        2019-02-20
            ...    
19471    2023-02-13
19472    2023-05-03
19473    2023-08-07
19474    2023-11-01
19475    2024-02-12
Name: trade_entry, Length: 19476, dtype: object

In [10]:
earnings_dates_for_tickers['trade_entry'] = pd.to_datetime(earnings_dates_for_tickers['trade_entry'])
earnings_dates_for_tickers['trade_entry']

0       2018-02-14
1       2018-05-14
2       2018-08-14
3       2018-11-19
4       2019-02-20
           ...    
19471   2023-02-13
19472   2023-05-03
19473   2023-08-07
19474   2023-11-01
19475   2024-02-12
Name: trade_entry, Length: 19476, dtype: datetime64[ns]

In [11]:
earnings_dates_for_tickers['trade_exit'] = pd.to_datetime(earnings_dates_for_tickers['trade_exit'])

# Extract only the date part (YYYY-MM-DD)
earnings_dates_for_tickers['trade_exit'] = earnings_dates_for_tickers['trade_exit'].dt.date
earnings_dates_for_tickers['trade_exit'] = pd.to_datetime(earnings_dates_for_tickers['trade_exit'])
earnings_dates_for_tickers['trade_exit']



0       2018-02-15
1       2018-05-15
2       2018-08-15
3       2018-11-20
4       2019-02-21
           ...    
19471   2023-02-14
19472   2023-05-04
19473   2023-08-08
19474   2023-11-02
19475   2024-02-13
Name: trade_exit, Length: 19476, dtype: datetime64[ns]

In [12]:
stock_prices.set_index(['date','ticker'],inplace = True)
stock_prices

,,permno,prc,bid,ask,comnam,permco
date,ticker,,,,,,
2018-02-07,A,87432,68.06,68.06000,68.07000,AGILENT TECHNOLOGIES INC,36364.0
2018-02-08,A,87432,65.05,65.03000,65.05000,AGILENT TECHNOLOGIES INC,36364.0
2018-02-09,A,87432,66.98,66.97000,66.98000,AGILENT TECHNOLOGIES INC,36364.0
2018-02-12,A,87432,68.43,68.40000,68.41000,AGILENT TECHNOLOGIES INC,36364.0
2018-02-13,A,87432,68.34,68.33000,68.34000,AGILENT TECHNOLOGIES INC,36364.0
...,...,...,...,...,...,...,...
2023-11-03,ZTS,13788,162.23,162.19000,162.23000,ZOETIS INC,54327.0
2023-11-06,ZTS,13788,163.17,163.20000,163.27000,ZOETIS INC,54327.0
2023-11-07,ZTS,13788,167.16,167.24001,167.28000,ZOETIS INC,54327.0


In [13]:
earnings_dates_for_tickers.head()

,Unnamed: 0,index,Short_ticker,Name,Ticker,Date,Event Type,Adjusted Trading Date,trade_entry,trade_exit
0,0,396,A,Agilent Technologies Inc,A US,2018-02-14 16:05:00,ER,2018-02-15 00:00:00+00:00,2018-02-14,2018-02-15
1,1,1298,A,Agilent Technologies Inc,A US,2018-05-14 16:05:00,ER,2018-05-15 00:00:00+00:00,2018-05-14,2018-05-15
2,2,2023,A,Agilent Technologies Inc,A US,2018-08-14 16:05:00,ER,2018-08-15 00:00:00+00:00,2018-08-14,2018-08-15
3,3,2754,A,Agilent Technologies Inc,A US,2018-11-19 16:05:00,ER,2018-11-20 00:00:00+00:00,2018-11-19,2018-11-20
4,4,3297,A,Agilent Technologies Inc,A US,2019-02-20 16:05:00,ER,2019-02-21 00:00:00+00:00,2019-02-20,2019-02-21


In [14]:
 
def entry_stock_price(row):
    try:
        # Check if there is an index match in stock_prices
        if (row['trade_entry'], row['Short_ticker']) in stock_prices.index:
            # Get the index position of the match
            match_index = stock_prices.index.get_loc((row['trade_entry'], row['Short_ticker']))
            # Get the previous 5 rows using slicing
            previous_rows = stock_prices.iloc[match_index-5:match_index+1]

            # get the 'prc' of the rows and return all of them as a list
            return previous_rows['prc'].to_list()
            
        else:
            return np.nan*6
    except KeyError:
        return np.nan*6

earnings_dates_for_tickers[['prc_5Dbefore', 'prc_4Dbefore', 'prc_3Dbefore', 'prc_2Dbefore','prc_1Dbefore','stock_price_entry']] = earnings_dates_for_tickers.apply(entry_stock_price, axis=1, result_type = 'expand')
        


In [15]:
def exit_stock_price(row):
    try:
        return stock_prices.loc[(row['trade_exit'], row['Short_ticker'])]['prc']
    except KeyError:
        return np.nan
earnings_dates_for_tickers['stock_price_exit'] = earnings_dates_for_tickers.apply(exit_stock_price,axis = 1)
      


In [16]:
earnings_dates_for_tickers['year_entry'] = earnings_dates_for_tickers['trade_entry'].dt.year

# Create 'year_exit' column containing the year of 'trade_exit'
earnings_dates_for_tickers['year_exit'] = earnings_dates_for_tickers['trade_exit'].dt.year
earnings_dates_for_tickers['year_entry']

0        2018
1        2018
2        2018
3        2018
4        2019
         ... 
19471    2023
19472    2023
19473    2023
19474    2023
19475    2024
Name: year_entry, Length: 19476, dtype: int32

In [17]:
options_dict[2018].columns

Index(['secid', 'symbol', 'cp_flag', 'expiry_indicator', 'volume',
       'strike_price', 'open_interest', 'impl_volatility', 'best_bid',
       'best_offer', 'delta', 'gamma', 'vega', 'theta'],
      dtype='object')

In [23]:
def straddle_iv_time(row):
    if row['year_entry'] < 2017 or row['year_entry'] > 2023 or row['year_exit'] < 2017 or row['year_exit'] > 2023:
        return [np.nan]*24
    try:

        IV_1 = []
        time_1 = []
        IV_2 = []
        time_2 = []

        # Trading Entry Date

        options = options_dict[row['year_entry']].loc[(row['Short_ticker'], row['trade_entry'])]

        # if there is no two exdates, add nan to the list
        exdates = options.index.get_level_values('exdate').unique().sort_values()
        if len(exdates) < 2:
            IV_1.append(np.nan)
            time_1.append(np.nan)
            IV_2.append(np.nan)
            time_2.append(np.nan)
        else:
            cloest_two_exdates = exdates[:2]

            # get the option with the closet exdate
            options_1 = options.loc[cloest_two_exdates[0]]
            options_1['abs_difference'] = abs(options_1['strike_price'] - row['stock_price_entry']*1000)

            # Filter the DataFrame for rows with the minimum absolute difference
            min_abs_difference = options_1['abs_difference'].min()
            filtered_options = options_1[options_1['abs_difference'] == min_abs_difference]

            if len(filtered_options) ==2 and ('C' in filtered_options['cp_flag'].values) and ('P' in filtered_options['cp_flag'].values):
                IV_1.append(filtered_options['impl_volatility'].mean())
                
                # calculate the calendar days between 'trade_entry' and closest_two_exdates[0]
                time_1.append((pd.to_datetime(cloest_two_exdates[0]) - row['trade_entry']).days)
            else:
                IV_1.append(np.nan)
                time_1.append(np.nan)

            # second closest exdate
            options_2 = options.loc[cloest_two_exdates[1]]
            options_2['abs_difference'] = abs(options_2['strike_price'] - row['stock_price_entry']*1000)

            # Filter the DataFrame for rows with the minimum absolute difference
            min_abs_difference = options_2['abs_difference'].min()
            filtered_options = options_2[options_2['abs_difference'] == min_abs_difference]

            if len(filtered_options) ==2 and ('C' in filtered_options['cp_flag'].values) and ('P' in filtered_options['cp_flag'].values):
                IV_2.append(filtered_options['impl_volatility'].mean())
                time_2.append((pd.to_datetime(cloest_two_exdates[1]) - row['trade_entry']).days)
            else:
                IV_2.append(np.nan)
                time_2.append(np.nan)


        # 1-5 days before earnings

        nyse = mcal.get_calendar('NYSE')
        current_date = row['trade_entry']
        schedule = nyse.schedule(start_date=current_date - pd.DateOffset(days=20), end_date=current_date)

        for i in range(1, 6):
            previous_trading_day = schedule.iloc[-(i+1)].name
            options = options_dict[row['year_entry']].loc[( row['Short_ticker'], previous_trading_day)]

            # if there is no two exdates, add nan to the list
            exdates = options.index.get_level_values('exdate').unique().sort_values()
            if len(exdates) < 2:
                IV_1.append(np.nan)
                time_1.append(np.nan)
                IV_2.append(np.nan)
                time_2.append(np.nan)
            else:

                cloest_two_exdates = exdates[:2]

                # get the option with the closet exdate
                options_1 = options.loc[cloest_two_exdates[0]]
                options_1['abs_difference'] = abs(options_1['strike_price'] - row[f'prc_{i}Dbefore']*1000)

                # Filter the DataFrame for rows with the minimum absolute difference
                min_abs_difference = options_1['abs_difference'].min()
                filtered_options = options_1[options_1['abs_difference'] == min_abs_difference]

                if len(filtered_options) ==2 and ('C' in filtered_options['cp_flag'].values) and ('P' in filtered_options['cp_flag'].values):
                    IV_1.append(filtered_options['impl_volatility'].mean())
                    
                    # calculate the calendar days between 'trade_entry' and closest_two_exdates[0]
                    time_1.append((pd.to_datetime(cloest_two_exdates[0]) - previous_trading_day).days)
                else:
                    IV_1.append(np.nan)
                    time_1.append(np.nan)

                # second closest exdate
                options_2 = options.loc[cloest_two_exdates[1]]
                options_2['abs_difference'] = abs(options_2['strike_price'] - row[f'prc_{i}Dbefore' ]*1000)

                # Filter the DataFrame for rows with the minimum absolute difference
                min_abs_difference = options_2['abs_difference'].min()
                filtered_options = options_2[options_2['abs_difference'] == min_abs_difference]

                if len(filtered_options) ==2 and ('C' in filtered_options['cp_flag'].values) and ('P' in filtered_options['cp_flag'].values):
                    IV_2.append(filtered_options['impl_volatility'].mean())
                    time_2.append((pd.to_datetime(cloest_two_exdates[1]) - previous_trading_day).days)
                else:
                    IV_2.append(np.nan)
                    time_2.append(np.nan)



        # return IV and time
        return IV_1 + time_1 + IV_2 + time_2

    except:
        return [np.nan]*24




earnings_dates_for_tickers[['straddle_iv_T1', 'straddle_iv_1d_T1','straddle_iv_2d_T1','straddle_iv_3d_T1','straddle_iv_4d_T1','straddle_iv_5d_T1',
                            'time_T1', 'time_1d_T1','time_2d_T1','time_3d_T1','time_4d_T1','time_5d_T1',
                            'straddle_iv_T2', 'straddle_iv_1d_T2','straddle_iv_2d_T2','straddle_iv_3d_T2','straddle_iv_4d_T2','straddle_iv_5d_T2',
                            'time_T2', 'time_1d_T2','time_2d_T2','time_3d_T2','time_4d_T2','time_5d_T2']] = earnings_dates_for_tickers.apply(straddle_iv_time, axis = 1, result_type = 'expand')

In [ ]:
# get 'trade_entry' unique date values which exclude year
unique_trade_entry = earnings_dates_for_tickers['trade_entry'].dt.date.unique()

# get only values in january
january_trade_entry = [date for date in unique_trade_entry if date.month == 1]

# in january, get the values that are in the first 5 days
january_1_5_trade_entry = [date for date in january_trade_entry if date.day <= 5]
january_1_5_trade_entry

[datetime.date(2018, 1, 4),
 datetime.date(2023, 1, 5),
 datetime.date(2022, 1, 5),
 datetime.date(2023, 1, 4),
 datetime.date(2024, 1, 3),
 datetime.date(2019, 1, 3),
 datetime.date(2020, 1, 3),
 datetime.date(2021, 1, 5),
 datetime.date(2018, 1, 2),
 datetime.date(2019, 1, 4),
 datetime.date(2024, 1, 5),
 datetime.date(2021, 1, 4),
 datetime.date(2024, 1, 4),
 datetime.date(2018, 1, 3),
 datetime.date(2020, 1, 2),
 datetime.date(2022, 1, 4),
 datetime.date(2019, 1, 2)]

In [ ]:
earnings_dates_for_tickers['stock_price_entry'].isna().sum()

5305

In [24]:
def straddle_iv_time(row):
    if row['Unnamed: 0'] % 100 == 0:
        print(row['Unnamed: 0'])
    if row['year_entry'] < 2017 or row['year_entry'] > 2023 or row['year_exit'] < 2017 or row['year_exit'] > 2023:
        return [np.nan]*24
    try:

        IV_1 = []
        time_1 = []
        IV_2 = []
        time_2 = []

        # Trading Entry Date

        options = options_dict[row['year_entry']].loc[(row['Short_ticker'], row['trade_entry'])]

        # if there is no two exdates, add nan to the list
        exdates = options.index.get_level_values('exdate').unique().sort_values()
        if len(exdates) < 2:
            return [np.nan]*24
        else:
            cloest_two_exdates = exdates[:2]

            # get the option with the closet exdate
            options_1 = options.loc[cloest_two_exdates[0]]
            options_1['abs_difference'] = abs(options_1['strike_price'] - row['stock_price_entry']*1000)

            # Filter the DataFrame for r
            # ows with the minimum absolute difference
            min_abs_difference = options_1['abs_difference'].min()
            filtered_options = options_1[options_1['abs_difference'] == min_abs_difference]

            if len(filtered_options) <=2:
                print (row['Unnamed: 0'])
                IV_1.append(filtered_options['impl_volatility'].mean())
                
                # calculate the calendar days between 'trade_entry' and closest_two_exdates[0]
                time_1.append((pd.to_datetime(cloest_two_exdates[0]) - row['trade_entry']).days)/365.0
            else:
                IV_1.append(np.nan)
                time_1.append(np.nan)

            # second closest exdate
            options_2 = options.loc[cloest_two_exdates[1]]
            options_2['abs_difference'] = abs(options_2['strike_price'] - row['stock_price_entry']*1000)

            # Filter the DataFrame for rows with the minimum absolute difference
            min_abs_difference = options_2['abs_difference'].min()
            filtered_options = options_2[options_2['abs_difference'] == min_abs_difference]

            if len(filtered_options) <=2:
                IV_2.append(filtered_options['impl_volatility'].mean())
                time_2.append((pd.to_datetime(cloest_two_exdates[1]) - row['trade_entry']).days)/365.0
            else:
                IV_2.append(np.nan)
                time_2.append(np.nan)


        # 1-5 days before earnings

            nyse = mcal.get_calendar('NYSE')
            current_date = row['trade_entry']
            schedule = nyse.schedule(start_date=current_date - pd.DateOffset(days=20), end_date=current_date)
            for i in range(1, 6):
                previous_trading_day = schedule.iloc[-(i+1)].name
                previous_year = previous_trading_day.year
                try:
                    options = options_dict[previous_year].loc[( row['Short_ticker'], previous_trading_day)]
                    # get the option with the closet exdate
                    options_1 = options.loc[cloest_two_exdates[0]]
                    options_1['abs_difference'] = abs(options_1['strike_price'] - row[f'prc_{i}Dbefore']*1000)

                        # Filter the DataFrame for rows with the minimum absolute difference
                    min_abs_difference = options_1['abs_difference'].min()
                    filtered_options = options_1[options_1['abs_difference'] == min_abs_difference]

                    if len(filtered_options) <= 2:
                        IV_1.append(filtered_options['impl_volatility'].mean())
                            
                            # calculate the calendar days between 'trade_entry' and closest_two_exdates[0]
                        time_1.append((pd.to_datetime(cloest_two_exdates[0]) - previous_trading_day).days)/365.0
                    else:
                        IV_1.append(np.nan)
                        time_1.append(np.nan)

                        # second closest exdate
                    options_2 = options.loc[cloest_two_exdates[1]]
                    options_2['abs_difference'] = abs(options_2['strike_price'] - row[f'prc_{i}Dbefore' ]*1000)

                        # Filter the DataFrame for rows with the minimum absolute difference
                    min_abs_difference = options_2['abs_difference'].min()
                    filtered_options = options_2[options_2['abs_difference'] == min_abs_difference]

                    if len(filtered_options) <=2:
                        IV_2.append(filtered_options['impl_volatility'].mean())
                        time_2.append((pd.to_datetime(cloest_two_exdates[1]) - previous_trading_day).days)/365.0
                    else:
                        IV_2.append(np.nan)
                        time_2.append(np.nan)
                except:
                    if len(IV_1) == i:
                        IV_1.append(np.nan)
                    if len(time_1) == i:
                        time_1.append(np.nan)
                    if len(IV_2) == i:
                        IV_2.append(np.nan)
                    if len(time_2) == i:
                        time_2.append(np.nan)

            # if there is no two exdates, add nan to the list
            


        # return IV and time
        return IV_1 + time_1 + IV_2 + time_2

    except:
        return [np.nan]*24


hundred = earnings_dates_for_tickers.head(100)

hundred[['straddle_iv_T1', 'straddle_iv_1d_T1','straddle_iv_2d_T1','straddle_iv_3d_T1','straddle_iv_4d_T1','straddle_iv_5d_T1',
                            'time_T1', 'time_1d_T1','time_2d_T1','time_3d_T1','time_4d_T1','time_5d_T1',
                            'straddle_iv_T2', 'straddle_iv_1d_T2','straddle_iv_2d_T2','straddle_iv_3d_T2','straddle_iv_4d_T2','straddle_iv_5d_T2',
                            'time_T2', 'time_1d_T2','time_2d_T2','time_3d_T2','time_4d_T2','time_5d_T2']] = hundred.apply(straddle_iv_time, axis = 1, result_type = 'expand')
hundred[~np.isnan(hundred['straddle_iv_T1'])]


0
0
1
2
25
26
27
28
52
54
55


,Unnamed: 0,index,Short_ticker,Name,Ticker,Date,Event Type,Adjusted Trading Date,trade_entry,trade_exit,...,straddle_iv_2d_T2,straddle_iv_3d_T2,straddle_iv_4d_T2,straddle_iv_5d_T2,time_T2,time_1d_T2,time_2d_T2,time_3d_T2,time_4d_T2,time_5d_T2


In [26]:
options_dict[2018].to_parquet("options_2018.parquet")